In [44]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import precision_score

import time
    
%matplotlib inline
import seaborn as sns

In [45]:
#Time Series Data
UR = pd.read_csv('UnempRate.csv', sep=',', na_values=[" ", ""])
crime = pd.read_csv('violent_crime_count_age17andup.csv', sep=',', na_values=[" ", ""])
#crime.iloc[:,1:]/population.iloc[:,1:] 
#but missing 2016 data, so 2016 data not used
#prescription is per 100
prescription = pd.read_csv('prescription_rate.csv', sep=',', na_values=[" ", ""])
population = pd.read_csv('population.csv', sep=',', na_values=[" ", ""])
birth_rate = pd.read_csv('birth_rate.csv', sep=',', na_values=[" ", ""])
death_rate = pd.read_csv('death_rate.csv', sep=',', na_values=[" ", ""])
children_poverty = pd.read_csv('children_living_in_poverty.csv', sep=',', na_values=[" ", ""])
teenage_pregnancy = pd.read_csv('Birth_Rate_by_10-19_Years_of_Age.csv', sep=',', na_values=[" ", ""])
drug_ER_Visits = pd.read_csv('ER_Visit_Rate_by_Residence-_Disorders_Related_to_Drug_Use.csv', sep=',', na_values=[" ", ""])

In [56]:
drug_ER_Visits

,County,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Appling,0.000400,0.000400,0.000395,0.000338,0.000508,0.000508,0.000508,0.000331,0.000712,0.000651,0.000599,0.000599,0.000378,0.001355,0.002822
1,Atkinson,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000837,0.000837,0.000837,0.000837,0.000000,0.000000,0.000000,0.000595,0.001692
2,Bacon,0.000488,0.000488,0.000488,0.000488,0.000000,0.000000,0.000000,0.000449,0.000449,0.001160,0.000536,0.000624,0.000624,0.002478,0.002374
3,Baker,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Baldwin,0.000265,0.000110,0.000174,0.000173,0.000215,0.000380,0.000209,0.000446,0.000372,0.000293,0.000345,0.000217,0.000327,0.001122,0.005183
5,Banks,0.000316,0.000316,0.000431,0.000539,0.000539,0.000342,0.000334,0.000384,0.000380,0.000548,0.000655,0.000326,0.000547,0.001244,0.003642
6,Barrow,0.000357,0.000421,0.000472,0.000449,0.000423,0.000695,0.000327,0.000438,0.000488,0.000472,0.000727,0.000644,0.000628,0.001725,0.003929
7,Bartow,0.000686,0.000537,0.000444,0.000675,0.000721,0.000470,0.000528,0.000721,0.000788,0.000548,0.000586,0.000573,0.000560,0.001635,0.005125
8,Ben Hill,0.000349,0.000643,0.000643,0.000290,0.000399,0.000395,0.000395,0.000395,0.000340,0.000340,0.000340,0.000685,0.000685,0.001207,0.003770
9,Berrien,0.000480,0.000294,0.000294,0.000294,0.000277,0.000277,0.000277,0.000277,0.000518,0.000518,0.000578,0.000578,0.000535,0.000791,0.003422


In [46]:
#Normalizing Time-Series Data
UR.iloc[:,1:]=UR.iloc[:,1:]/100
teenage_pregnancy.iloc[:,1:]=teenage_pregnancy.iloc[:,1:]/1000
drug_ER_Visits.iloc[:,1:]=drug_ER_Visits.iloc[:,1:]/100000
birth_rate.iloc[:,1:]=birth_rate.iloc[:,1:]/1000
death_rate.iloc[:,1:]=death_rate.iloc[:,1:]/1000
prescription.iloc[:,1:]=prescription.iloc[:,1:]/100

In [47]:
#Non-Time Series Data
data = pd.read_csv('cleaned_GaCountiesODDeath.csv', na_values=[" ", ""] ,sep=',', index_col=0).sort('County').reset_index(drop=True)
hospital = pd.read_csv('cleaned_GaHospitals.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
dropbox = pd.read_csv('drug_dropbox.csv', na_values=[" ", ""] ,sep=',').fillna(0)
ems = pd.read_csv('cleaned_ems.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
rehab = pd.read_csv('cleaned_rehab.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)

hospital=hospital.rename(columns = {'Count':'hospital_count'})
dropbox=dropbox.rename(columns = {'COUNT':'dropbox_count'})
dropbox=dropbox.rename(columns = {'24Availability':'dropbox_247availability'})

In [48]:
normalized_data = data.iloc[:,1:]/population.iloc[:,1:]

In [49]:
df = data[['County','Rural']].copy()
df1 = pd.concat([df, rehab['Rehab_Count'],  ems['EMS_COUNT'],
                 hospital['hospital_count'], dropbox['dropbox_247availability'], dropbox['dropbox_count']], axis=1)

In [51]:
#data[['Counties','Rural', '2016', '1yrlag', '2yrlag', '10yrlag']]
df1['1yrlag']=(data['2016']-data['2015'])/data['2015']
df1['5yrlag']=(data['2016']-data['2011'])/data['2011']
df1['10yrlag']=(data['2016']-data['2006'])/data['2006']

df1['UR_1yrlag']=(UR['2016']-UR['2015'])/UR['2015']
df1['UR_5yrlag']=(UR['2016']-UR['2011'])/UR['2011']
df1['UR_10yrlag']=(UR['2016']-UR['2006'])/UR['2006']

df1['CR_1yrlag']=(crime['2015']-crime['2014'])/crime['2014']
df1['CR_5yrlag']=(crime['2015']-crime['2010'])/crime['2010']
df1['CR_10yrlag']=(crime['2015']-crime['2005'])/crime['2005']

df1['PR_1yrlag']=(prescription['2016']-prescription['2015'])/prescription['2015']
df1['PR_5yrlag']=(prescription['2016']-prescription['2011'])/prescription['2011']
df1['PR_10yrlag']=(prescription['2016']-prescription['2006'])/prescription['2006']

df1['BR_1yrlag']=(birth_rate['2016']-birth_rate['2015'])/birth_rate['2015']
df1['BR_5yrlag']=(birth_rate['2016']-birth_rate['2011'])/birth_rate['2011']
df1['BR_10yrlag']=(birth_rate['2016']-birth_rate['2006'])/birth_rate['2006']

df1['DR_1yrlag']=(death_rate['2016']-death_rate['2015'])/death_rate['2015']
df1['DR_5yrlag']=(death_rate['2016']-death_rate['2011'])/death_rate['2011']
df1['DR_10yrlag']=(death_rate['2016']-death_rate['2006'])/death_rate['2006']

df1['CP_1yrlag']=(children_poverty['2015']-children_poverty['2014'])/children_poverty['2014']
df1['CP_5yrlag']=(children_poverty['2015']-children_poverty['2010'])/children_poverty['2010']
df1['CP_10yrlag']=(children_poverty['2015']-children_poverty['2005'])/children_poverty['2005']

df1['TP_1yrlag']=(teenage_pregnancy['2016']-teenage_pregnancy['2015'])/teenage_pregnancy['2015']
df1['TP_5yrlag']=(teenage_pregnancy['2016']-teenage_pregnancy['2011'])/teenage_pregnancy['2011']
df1['TP_10yrlag']=(teenage_pregnancy['2016']-teenage_pregnancy['2006'])/teenage_pregnancy['2006']

df1['ER_1yrlag']=(drug_ER_Visits['2016']-drug_ER_Visits['2015'])/drug_ER_Visits['2015']
df1['ER_5yrlag']=(drug_ER_Visits['2016']-drug_ER_Visits['2011'])/drug_ER_Visits['2011']
df1['ER_10yrlag']=(drug_ER_Visits['2016']-drug_ER_Visits['2006'])/drug_ER_Visits['2006']

df1=df1.replace([np.inf, -np.inf], np.nan)
df1 = df1.fillna(0)

drug_ER_Visits_2016=drug_ER_Visits.rename(columns = {'2016':'drug_ER_Visits_2016'})
teenage_pregnancy_2016=teenage_pregnancy.rename(columns = {'2016':'teenage_pregnancy_2016'})
children_poverty_2015=children_poverty.rename(columns = {'2015':'2015_children_poverty'})
death_rate_2016=death_rate.rename(columns = {'2016':'2016_death_rate'})
birth_rate_2016=birth_rate.rename(columns = {'2016':'2016_birth_rate'})
prescription_2016=prescription.rename(columns = {'2016':'2016_prescription'})

df2 = pd.concat([df1, drug_ER_Visits_2016['drug_ER_Visits_2016'], 
                 teenage_pregnancy_2016['teenage_pregnancy_2016'], 
                 children_poverty_2015['2015_children_poverty'], 
                 death_rate_2016['2016_death_rate'], birth_rate_2016['2016_birth_rate'], 
                 prescription_2016['2016_prescription'], normalized_data['2016']], axis=1)
df2=df2.rename(columns = {'2016':'2016_ODRate'})

In [52]:
df2.columns
nat_OD_rate=0.00016219127

In [53]:
#2015 National OD Rate
#52,404/323100000
df2['2016ODabovenatavg'] = df2['2016_ODRate'] > nat_OD_rate

In [55]:
df2.to_csv('AggredgatedData.csv')

In [54]:
df2

,County,Rural,Rehab_Count,EMS_COUNT,hospital_count,dropbox_247availability,dropbox_count,1yrlag,5yrlag,10yrlag,...,ER_5yrlag,ER_10yrlag,drug_ER_Visits_2016,teenage_pregnancy_2016,2015_children_poverty,2016_death_rate,2016_birth_rate,2016_prescription,2016_ODRate,2016ODabovenatavg
0,Appling,1,0,1,0,1,1,0.000000,0.000000,0.000000,...,3.334869,4.555118,0.002822,0.0285,0.342,0.011610,0.013563,1.237,0.000163,True
1,Atkinson,1,0,0,0,0,0,0.000000,0.000000,0.000000,...,1.021505,0.000000,0.001692,0.0223,0.405,0.007339,0.017927,0.679,0.000121,False
2,Bacon,1,0,1,0,0,1,0.000000,-1.000000,-1.000000,...,1.046552,0.000000,0.002374,0.0190,0.368,0.009623,0.012714,1.575,0.000000,False
3,Baker,1,0,1,0,0,1,-1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0324,0.419,0.008558,0.010143,0.000,0.000000,False
4,Baldwin,0,1,1,2,1,1,-1.000000,-1.000000,-1.000000,...,16.689420,23.106977,0.005183,0.0074,0.355,0.009981,0.009760,0.984,0.000000,False
5,Banks,1,0,1,0,0,1,0.333333,-0.333333,3.000000,...,5.645985,5.756957,0.003642,0.0142,0.244,0.008355,0.011718,0.423,0.000217,True
6,Barrow,0,1,1,1,0,2,0.444444,0.625000,0.444444,...,7.324153,8.288416,0.003929,0.0082,0.213,0.007725,0.014110,0.846,0.000169,True
7,Bartow,0,1,1,1,1,2,0.105263,0.235294,1.333333,...,8.352190,6.108183,0.005125,0.0134,0.215,0.008549,0.012581,1.029,0.000202,True
8,Ben Hill,1,0,1,1,1,1,0.000000,0.000000,0.000000,...,10.088235,8.448622,0.003770,0.0167,0.469,0.012949,0.013643,1.994,0.000058,False
9,Berrien,1,1,1,1,0,1,0.000000,0.000000,-0.666667,...,5.606178,11.353791,0.003422,0.0309,0.369,0.011381,0.013857,0.561,0.000053,False
